In [1]:
# Importing Packages 

import numpy as np
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [95]:
# Importing Text

data = pd.read_csv("Uber_Category_3000.csv",encoding='latin1')

In [96]:
data.head()

,Tweets,Category
0,hi we understand this can be upsetting driver ...,1
1,hey savani weve fixed this for you and process...,1
2,hey sanju for us to assist you better kindly h...,1
3,we understand your concern nikhil we have made...,0
4,hey vinay were sorry to hear about the trouble...,1


In [97]:
# Cleaning Text

df_clean = data
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['Tweets'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)

In [98]:
df_clean.head()

,Tweets,Category,clean,tokens
0,hi we understand this can be upsetting driver ...,1,hi we understand this can be upsetting driver ...,"[hi, we, understand, this, can, be, upsetting,..."
1,hey savani weve fixed this for you and process...,1,hey savani weve fixed this for you and process...,"[hey, savani, weve, fixed, this, for, you, and..."
2,hey sanju for us to assist you better kindly h...,1,hey sanju for us to assist you better kindly h...,"[hey, sanju, for, us, to, assist, you, better,..."
3,we understand your concern nikhil we have made...,0,we understand your concern nikhil we have made...,"[we, understand, your, concern, nikhil, we, ha..."
4,hey vinay were sorry to hear about the trouble...,1,hey vinay were sorry to hear about the trouble...,"[hey, vinay, were, sorry, to, hear, about, the..."


In [7]:
# Importing 

import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [99]:
import multiprocessing
from gensim.models import Word2Vec

# WORD2VEC()
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()

w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)


#TRAIN()

w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)

(107028705, 941840000)

In [11]:
#words similar to thanks
w2v_model.wv.most_similar(positive=["understand"])

#Words similar to please
w2v_model.wv.most_similar(positive=["please"])

[('via', 0.37331557273864746),
 ('message', 0.3613654673099518),
 ('direct', 0.34762924909591675),
 ('this', 0.3421033024787903),
 ('email', 0.32505300641059875),
 ('registered', 0.3240955173969269),
 ('help', 0.3181191682815552),
 ('can', 0.31372693181037903),
 ('number', 0.3121623396873474),
 ('also', 0.3105176091194153)]

In [100]:
#First defining the X (input), and the y (output)

y = data['Category'].values
X = np.array(df_clean["tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)

In [14]:
vectorizer = CountVectorizer()

In [21]:
vectorizer

TfidfVectorizer(analyzer=<function <lambda> at 0x00000147A1C4F948>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [102]:
X_train = vectorizer.fit_transform(X_train)

In [103]:
X_test = vectorizer.transform(X_test)

In [104]:
feature_names = vectorizer.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Number of features: 391


In [105]:
scores = cross_val_score(RandomForestClassifier(), X_train, y_train, cv=5)
print("Mean Cross-Validation accuracy: {:.2f}".format(np.mean(scores)))

Mean Cross-Validation accuracy: 0.95


In [106]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Testing set score: {:.3f}".format(logreg.score(X_test, y_test)))

Training set score: 0.931
Testing set score: 0.927


In [107]:
pred_logreg = logreg.predict(X_test)
confusion = confusion_matrix(y_test, pred_logreg)
print("Confusion Matrix: \n{}".format(confusion))

Confusion Matrix: 
[[ 45  42]
 [  2 511]]


In [108]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
print("Training set score: {:.3f}".format(nb.score(X_train, y_train)))
print("Testing set score: {:.3f}".format(nb.score(X_test, y_test)))

Training set score: 0.877
Testing set score: 0.878


In [109]:
pred_nb = nb.predict(X_test)
confusion = confusion_matrix(y_test, pred_nb)
print("Confusion Matrix: \n{}".format(confusion))

Confusion Matrix: 
[[ 14  73]
 [  0 513]]


In [110]:
rf = RandomForestClassifier(n_estimators=100, bootstrap= True, max_features = 'sqrt')
rf.fit(X_train, y_train)
print("Training set score: {:.3f}".format(rf.score(X_train, y_train)))
print("Testing set score: {:.3f}".format(rf.score(X_test, y_test)))

Training set score: 1.000
Testing set score: 0.963


In [111]:
pred_rf = rf.predict(X_test)
confusion = confusion_matrix(y_test, pred_rf)
print("Confusion Matrix: \n{}".format(confusion))

Confusion Matrix: 
[[ 68  19]
 [  3 510]]
